Below is a basic webscraping script designed for personal use.

This requires 3 user entries (keywords, phrase and location) to filter job search results.
This will save all the job details from all the search results pages in a table format and save it locally as a csv file.

Please note that there is no error handling in the user input phase as I built this for personal use (considering only job queries in BC, Canada).

In [ ]:
#Return an integer for values under Post Date for easy sorting later on
def get_days_ago(date):
    if date=="Today":
        return 0
    else:
        return int(date[:2])

In [ ]:
#Extracts job details from html tags and returns a dictionary
def parse_job_details(item):
    jobdetails={}
    jobdetails["Job Title"]=item.find("div",{"class":"title"}).text.replace("\n","")
    jobdetails["Company"]=item.find("span",{"class":"company"}).text.replace("\n","")
    try:
        jobdetails["Location"]=item.find("span",{"class":"location accessible-contrast-color-location"}).text
        
    except:
        jobdetails["Location"]=""
    try:
        jobdetails["Salary"]=item.find("span",{"class":"salaryText"}).text.replace("\n","")
    except:
        jobdetails["Salary"]=""
    jobdetails["Summary"]=item.find("div",{"class":"summary"}).text.replace("\n","")
    jobdetails["Post Date"]=get_days_ago(item.find("span",{"class":"date"}).text.replace("\n",""))
    try:
        jobdetails["Easy Apply"]=item.find("span",{"class":"iaLabel"}).text.replace("\n","")
    except:
        jobdetails["Easy Apply"]=""
    jobdetails["Page URL"]="https://ca.indeed.com"+item.find("a").get('href')
    return jobdetails

In [ ]:
import requests
from bs4 import BeautifulSoup

#Get user input
keywords=input("Enter job title, keywords, or company : ")
phrase=input("With the exact phrase : ")
location=input("Enter city in BC, Canada: ")

#Establish the url string based on user input
base_url="https://ca.indeed.com/jobs?q="+keywords.replace(" ","+")+"&as_phr="+phrase.replace(" ","+")+"&l="+location.strip()+"%2C+BC"
base_url2="&start="

#Checking the first page of the search results
loadpage= requests.get(base_url,headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
pagecon = loadpage.content
soup=BeautifulSoup(pagecon,"html.parser")

#Find out how many pages are there in the search results
page=soup.find_all("span",{"class":"pn"})
lastpage=len(page)

#For each page in Indeed.com search results, the value after "&start=" is incremented by 10
increment=10

In [ ]:
results=[] #main table where all job details are saved

for page in range(0,lastpage):
    url = base_url+base_url2+str(page*increment)

    #Request info for each page in the search results
    callpage=requests.get(url,headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    savepage=callpage.content
    soup=BeautifulSoup(savepage,"html.parser")

    #Defines the webscraping scope
    all=soup.find_all("div",{"class":"jobsearch-SerpJobCard unifiedRow row result"})   

    #Process each item in soup                          
    for item in all:
        indeed = parse_job_details(item) #returns a dictionary 
        results.append(indeed) #job details are saved as a row in the list


In [ ]:
#Converts the dictionary to a dataframe
import pandas
indeed_df=pandas.DataFrame(results)
indeed_df.sort_values(by=['Post Date'], inplace=True,ascending=True)


#Displays the dataframe (showing all job details in a table format)
indeed_df

In [ ]:
#Saves the table locally in a csv format with the filename "Indeed"
indeed_df.to_csv("Indeed.csv")